In [14]:
import pandas as pd
import sqlite3

import plotly.io as pio
pio.renderers.default="iframe"

maindf = pd.read_csv("data_2022_2023.csv")
maindf


,LastName,FirstName,Gender,Country,Date,Competition,Round,Location,Apparatus,Rank,D_Score,E_Score,Penalty,Score
0,AAS,Fredrik,m,NOR,24-27 Feb 2022,2022 Cottbus World Cup,qual,"Cottbus, Germany",HB,18.0,3.9,8.266,NaN,12.166
1,AAS,Fredrik,m,NOR,24-27 Feb 2022,2022 Cottbus World Cup,qual,"Cottbus, Germany",PB,23.0,3.9,6.900,NaN,10.800
2,AAS,Fredrik,m,NOR,24-27 Feb 2022,2022 Cottbus World Cup,qual,"Cottbus, Germany",PH,33.0,4.2,6.666,NaN,10.866
3,AAS,Fredrik,m,NOR,23-26 Feb 2023,2023 Cottbus World Cup,qual,"Cottbus, Germany",HB,39.0,4.6,6.700,NaN,11.300
4,AAS,Fredrik,m,NOR,23-26 Feb 2023,2023 Cottbus World Cup,qual,"Cottbus, Germany",PH,44.0,4.4,7.800,NaN,12.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24429,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,final,"Tel Aviv, Israel",FX,8.0,4.8,7.050,NaN,11.850
24430,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,qual,"Tel Aviv, Israel",FX,3.0,5.8,7.950,0.1,13.650
24431,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,qual,"Tel Aviv, Israel",HB,10.0,5.0,6.250,NaN,11.250
24432,ÖNDER,Ahmet,m,TUR,1-4 Jun 2023,2023 Tel Aviv World Challenge Cup,final,"Tel Aviv, Israel",PB,1.0,6.3,8.050,NaN,14.350


In [15]:
import inspect
from sqlPlots import data_cleaning
print(inspect.getsource(data_cleaning)) 

maindf = data_cleaning(maindf)
print(maindf)

def data_cleaning(maindf):
    maindf.dropna(inplace=True, subset=[ 'Apparatus', 'Score', 'Country', 'D_Score', "E_Score"])
    maindf.loc[maindf['Apparatus'] == 'VT1', 'Apparatus'] = 'VT'
    maindf.loc[maindf['Apparatus'] == 'VT2', 'Apparatus'] = 'VT'
    maindf = maindf[maindf['Gender'] == 'w']
    return maindf

         LastName  FirstName Gender Country                      Date  \
144    ABDELSALAM       Jana      w     EGY  29 Oct 2022 - 6 Nov 2022   
145    ABDELSALAM       Jana      w     EGY  29 Oct 2022 - 6 Nov 2022   
146    ABDELSALAM       Jana      w     EGY  29 Oct 2022 - 6 Nov 2022   
147    ABDELSALAM       Jana      w     EGY  29 Oct 2022 - 6 Nov 2022   
148    ABDELSALAM       Jana      w     EGY            17-20 Mar 2022   
...           ...        ...    ...     ...                       ...   
24412     ZWICKER  Christina      w     CRO            11-16 Apr 2023   
24413     ZWICKER  Christina      w     CRO              1-4 Jun 2023   
24414     ZWICKER  Christ

In [16]:
import plotly.express as px

from sqlPlots import difficultyVsExecutionPlot
print(inspect.getsource(difficultyVsExecutionPlot)) 

difficultyVsExecutionPlot(maindf)


def difficultyVsExecutionPlot(maindf):
    fig = px.scatter(maindf, 
                  x="D_Score", 
                  y="E_Score", 
                  color="Apparatus", 
                  hover_data=['LastName', 'FirstName'],
                  title="Difficulty vs Execution Tradeoff Across Apparatuses",
                  trendline="ols")

    fig.update_layout(
        xaxis_title="Average Difficulty Score (D_Score)", 
        yaxis_title="Average Execution Score (E_Score)",
        legend_title="Apparatus",
    )

    fig.show()



In [17]:
with sqlite3.connect("gym") as conn:
    maindf.to_sql("gym", conn, if_exists = "replace", index = False) 
    

In [18]:
from sqlPlots import query_gym_country_database
print(inspect.getsource(query_gym_country_database)) 

query_gym_country_database("NOR")


def query_gym_country_database(country):
    with sqlite3.connect('gym') as conn:
        cmd = \
        f'''
        SELECT LastName, FirstName, Apparatus, AVG(Score) AS PredictedScore, MAX(Score) as Maxscore, 
        SQRT(AVG(score * score) - AVG(score) * AVG(score)) AS StdDevScore,
        COUNT(DISTINCT Date) AS CompetitionsCount
        FROM gym 
        WHERE Country = '{country}'
        GROUP BY LastName, FirstName, Apparatus 
        ORDER BY Apparatus, Maxscore DESC
        '''
    df = pd.read_sql_query(cmd, conn)
    return (df)



,LastName,FirstName,Apparatus,PredictedScore,Maxscore,StdDevScore,CompetitionsCount
0,TRONRUD,Maria,BB,12.333000,13.233,0.525935,10
1,MADSØ,Julie,BB,12.249750,12.633,0.284493,2
2,NEURAUTER,Mali,BB,11.507750,12.366,0.586156,4
3,ROENBECK,Marie,BB,11.319800,12.266,0.722424,4
4,KANTER,Mari,BB,11.445500,12.233,0.931899,7
5,TØSSEBRO,Juliane,BB,12.050000,12.100,0.050000,2
6,TOESSEBRO,Juliane,BB,11.388667,11.800,0.315591,3
7,MADSOE,Julie,BB,11.566000,11.566,0.000000,1
8,HALVORSEN,Selma,BB,9.900000,9.900,0.000000,1
9,TRONRUD,Maria,FX,11.375714,12.666,0.930657,6


In [19]:
import plotly
from plotly import express as px
import plotly.io as pio

from sqlPlots import scatterplot_by_country
print(inspect.getsource(scatterplot_by_country)) 
scatterplot_by_country("USA")

def scatterplot_by_country(country):
    fig = px.scatter(query_gym_country_database(country),
                x = "Apparatus", 
                y = "PredictedScore", 
                color="LastName",
                size='CompetitionsCount', hover_data=['PredictedScore'])


    fig.update_layout(
        title=f"Scatterplot of Gymnasts' MaxScore in Country '{country}'", #The colorbar and overall plot have professional titles.
        yaxis_title="Max Score of Gymnasts",
        )


    fig.show()



import pandas as pd
df = pd.read_csv("data_2022_2023.csv")
print(df.head())

In [20]:
import sqlite3
import pandas as pd

from sqlPlots import query_gym_database
print(inspect.getsource(query_gym_database)) 

pivoted_table = query_gym_database()
(pivoted_table)


def query_gym_database():
    with sqlite3.connect('gym') as conn:
        cmd = f'''
        WITH AthleteScores AS (
            SELECT 
                LastName, 
                FirstName,
                Apparatus,
                AVG(Score) AS PredictedScore,
                COUNT(DISTINCT Date) AS CompetitionsCount,
                Country
            FROM gym 
            GROUP BY LastName, FirstName, Apparatus
        )
        SELECT 
            LastName, 
            FirstName,
            Country,
            MAX(CASE WHEN Apparatus = 'BB' THEN PredictedScore END) AS BB_PredictedScore,
            MAX(CASE WHEN Apparatus = 'VT' THEN PredictedScore END) AS VT_PredictedScore,
            MAX(CASE WHEN Apparatus = 'FX' THEN PredictedScore END) AS FX_PredictedScore,
            MAX(CASE WHEN Apparatus = 'UB' THEN PredictedScore END) AS UB_PredictedScore
          
        FROM AthleteScores
        GROUP BY LastName, FirstName, Country
        ORDER BY LastName, FirstName, Coun

,LastName,FirstName,Country,BB_PredictedScore,VT_PredictedScore,FX_PredictedScore,UB_PredictedScore
0,ABDELSALAM,Jana,EGY,11.678750,12.3665,11.50800,10.775857
1,ABDULLAHI,Ayesha,GBR,10.200000,NaN,NaN,10.200000
2,ABEYRATNE,Kumudi Imanya,SRI,9.100000,NaN,7.35000,5.500000
3,ABOELHASAN,Jana,EGY,10.500000,11.6000,11.45800,10.900000
4,ABREU,Yamilet,DOM,10.100000,12.5670,12.36700,11.733000
...,...,...,...,...,...,...,...
856,ZIVADINOVIC,Kristina,SRB,9.333167,NaN,11.35825,NaN
857,ZLOBEC,Evandra,CAN,11.200000,11.9415,12.65000,11.933500
858,ZONNEVELD,Maya,CAN,NaN,NaN,NaN,10.550000
859,ZUO,Tong,CHN,13.288667,12.8330,13.16600,13.819600


In [21]:
from sqlPlots import query_gym_database
print(inspect.getsource(query_gym_database)) 

def query_gym_database():
    with sqlite3.connect('gym') as conn:
        cmd = f'''
        WITH AthleteScores AS (
            SELECT 
                LastName, 
                FirstName,
                Apparatus,
                AVG(Score) AS PredictedScore,
                COUNT(DISTINCT Date) AS CompetitionsCount,
                Country
            FROM gym 
            GROUP BY LastName, FirstName, Apparatus
        )
        SELECT 
            LastName, 
            FirstName,
            Country,
            MAX(CASE WHEN Apparatus = 'BB' THEN PredictedScore END) AS BB_PredictedScore,
            MAX(CASE WHEN Apparatus = 'VT' THEN PredictedScore END) AS VT_PredictedScore,
            MAX(CASE WHEN Apparatus = 'FX' THEN PredictedScore END) AS FX_PredictedScore,
            MAX(CASE WHEN Apparatus = 'UB' THEN PredictedScore END) AS UB_PredictedScore
          
        FROM AthleteScores
        GROUP BY LastName, FirstName, Country
        ORDER BY LastName, FirstName, Coun

In [22]:
import numpy as np
from simulations import add_user_entry
print(inspect.getsource(add_user_entry)) 
add_user_entry(pivoted_table)
print(pivoted_table)

def add_user_entry(df):
    index = len(df)
    df.loc[index] = ['Test', 'Test','USA', 15, 15, 15, 15]
    print(df)

       LastName      FirstName Country  BB_PredictedScore  VT_PredictedScore  \
0    ABDELSALAM           Jana     EGY          11.678750            12.3665   
1     ABDULLAHI         Ayesha     GBR          10.200000                NaN   
2     ABEYRATNE  Kumudi Imanya     SRI           9.100000                NaN   
3    ABOELHASAN           Jana     EGY          10.500000            11.6000   
4         ABREU        Yamilet     DOM          10.100000            12.5670   
..          ...            ...     ...                ...                ...   
857      ZLOBEC        Evandra     CAN          11.200000            11.9415   
858   ZONNEVELD           Maya     CAN                NaN                NaN   
859         ZUO           Tong     CHN          13.288667            12.8330   
860     ZWICKER      Christina     CRO          12.083167            11.5000   
86

In [23]:
import inspect
from simulations import delete_recent_entry
print(inspect.getsource(delete_recent_entry)) 
delete_recent_entry(pivoted_table)

def delete_recent_entry(df):
    index = len(df)
    df = df.drop(index - 1)
    print(df)

        LastName      FirstName Country  BB_PredictedScore  VT_PredictedScore  \
0     ABDELSALAM           Jana     EGY          11.678750            12.3665   
1      ABDULLAHI         Ayesha     GBR          10.200000                NaN   
2      ABEYRATNE  Kumudi Imanya     SRI           9.100000                NaN   
3     ABOELHASAN           Jana     EGY          10.500000            11.6000   
4          ABREU        Yamilet     DOM          10.100000            12.5670   
..           ...            ...     ...                ...                ...   
856  ZIVADINOVIC       Kristina     SRB           9.333167                NaN   
857       ZLOBEC        Evandra     CAN          11.200000            11.9415   
858    ZONNEVELD           Maya     CAN                NaN                NaN   
859          ZUO           Tong     CHN          13.288667            12.8330   
860      ZWICKER 

In [26]:
import inspect
import numpy as np
from simulations import monte_carlo
print(inspect.getsource(monte_carlo)) 

pivoted_table = monte_carlo(pivoted_table)

def monte_carlo(df):
    list_of_events = ['BB', 'VT', 'FX', 'UB']

    num_simulations = 1000

    df['gold'] = 0
    df['silver'] = 0
    df['bronze'] = 0

    for i in range(num_simulations): 
        for event in list_of_events:
            event_data = df[df[f'{event}_PredictedScore'].notna()].copy() #only select data that is the specific event type
            event_data['simulated_score'] = event_data[f'{event}_PredictedScore'] + np.random.normal(0, 0.1, size=len(event_data)) #add noise to create simulated score
            event_data = event_data.sort_values(by='simulated_score', ascending=False).reset_index(drop=True) #sort simulated scores from highest to lowest

            #award medals to top three scorers in simulated score
            df.loc[df['LastName'] == event_data.loc[0, 'LastName'], 'gold'] += 1
            df.loc[df['LastName'] == event_data.loc[1, 'LastName'], 'silver'] += 1
            df.loc[df['LastName'] == event_data.loc[2, 'LastName'], 'bronze'] += 1

    

In [27]:
import inspect
from simulations import medal_count_by_country
print(inspect.getsource(medal_count_by_country)) 
medal_count_by_country(pivoted_table, "USA")


def medal_count_by_country(df, country):
    results_by_country = (df[df['Country']==country]).head()
    results_by_country

    fig = px.histogram(results_by_country, 
                    x="LastName", 
                    y=["gold", 'silver', 'bronze'], 
                    title=f"Medal County per Athele for Country '{country}'",
                    color_discrete_sequence=['gold', 'silver', '#CD7F32']
                    )

    fig.update_layout(
        xaxis_title="Gymnast Last Name", 
        yaxis_title="Predicted Olympic Medal Count out of 1000 Simulations",
        legend_title="Medal Type",
    )

    fig.show()

